# Data Preprocessing : Demonstrated removing useless varaiables, EDA, imputation and encoding

In [480]:
# Importing the required libraries 

import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from math import sqrt 
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [481]:
# Importing Data Sets
male_legacy = pd.read_csv("male_players (legacy).csv")

male_legacy.info()

C:\Users\logan.anabi\AppData\Local\Temp\ipykernel_23384\1089976023.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  male_legacy = pd.read_csv("male_players (legacy).csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [482]:
male_legacy.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [483]:
male_legacy.describe()

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,161583.000000,161583.000000,161583.0,161583.000000,161583.000000,1.595300e+05,159822.000000,161583.000000,161583.000000,161583.000000,...,128722.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,161583.000000,161583.000000,161583.000000,17969.000000
mean,214484.722353,19.125514,2.0,65.699071,70.744008,2.326770e+06,10855.409768,25.123181,181.240205,75.235031,...,57.816892,45.757957,47.669996,45.698588,16.52961,16.274918,16.140374,16.288861,16.636973,39.149090
std,34928.608856,2.559318,0.0,7.040855,6.259121,6.005746e+06,21941.656285,4.670207,6.750148,7.000456,...,12.329739,20.453699,21.336404,20.935273,17.67047,16.834294,16.476466,16.998697,17.980143,10.503788
min,2.000000,15.000000,2.0,40.000000,40.000000,1.000000e+03,500.000000,16.000000,154.000000,49.000000,...,3.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,199159.000000,17.000000,2.0,61.000000,66.000000,3.250000e+05,2000.000000,21.000000,176.000000,70.000000,...,50.000000,26.000000,27.000000,25.000000,8.00000,8.000000,8.000000,8.000000,8.000000,31.000000
50%,220621.000000,19.000000,2.0,66.000000,70.000000,7.250000e+05,4000.000000,25.000000,181.000000,75.000000,...,59.000000,50.000000,54.000000,52.000000,11.00000,11.000000,11.000000,11.000000,11.000000,41.000000
75%,236958.000000,21.000000,2.0,70.000000,75.000000,1.800000e+06,10000.000000,28.000000,186.000000,80.000000,...,66.000000,63.000000,66.000000,64.000000,14.00000,14.000000,14.000000,14.000000,14.000000,46.000000
max,271817.000000,23.000000,2.0,94.000000,95.000000,1.940000e+08,575000.000000,54.000000,208.000000,110.000000,...,96.000000,94.000000,94.000000,95.000000,91.00000,92.000000,95.000000,92.000000,94.000000,68.000000


In [484]:
#Function for data preprocessing 
def preprocess_data(data):
    # Drop columns with more than 30% missing values
    threshold = 0.3 * data.shape[0]
    data = data[data.columns[data.isnull().sum() < threshold]]

    # Split data into numeric and non-numeric
    numeric_data = data.select_dtypes(include=np.number)
    categorical_data = data.select_dtypes(include=['object'])

    # Convert categorical data to numerical data
    for column in categorical_data.columns:
        categorical_data[column], _ = pd.factorize(categorical_data[column])

    # Drop unnecessary columns
    columns_to_drop = ['player_url', 'player_face_url', 'long_name', 'real_face', 'short_name','dob','wage_eur','value_eur']
    categorical_data = categorical_data.drop(columns_to_drop, axis=1, errors='ignore')

    # Fill NaN values with the mean
    categorical_data = categorical_data.fillna(categorical_data.mean())
    numeric_data = numeric_data.fillna(numeric_data.mean())

    # Combine cleaned data
    cleaned_data = pd.concat([categorical_data, numeric_data], axis=1)

    return cleaned_data


In [485]:
# Load and preprocess training data
cleaned_data = preprocess_data(male_legacy)

Y = cleaned_data['overall']
X = cleaned_data.drop('overall', axis=1)

In [486]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 96 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   fifa_update_date                161583 non-null  int64  
 1   player_positions                161583 non-null  int64  
 2   league_name                     161583 non-null  int64  
 3   club_name                       161583 non-null  int64  
 4   club_position                   161583 non-null  int64  
 5   club_joined_date                161583 non-null  int64  
 6   nationality_name                161583 non-null  int64  
 7   preferred_foot                  161583 non-null  int64  
 8   work_rate                       161583 non-null  int64  
 9   body_type                       161583 non-null  int64  
 10  ls                              161583 non-null  int64  
 11  st                              161583 non-null  int64  
 12  rs              

In [487]:
Y = cleaned_data['overall']
X = cleaned_data.drop('overall',axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 95 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   fifa_update_date                161583 non-null  int64  
 1   player_positions                161583 non-null  int64  
 2   league_name                     161583 non-null  int64  
 3   club_name                       161583 non-null  int64  
 4   club_position                   161583 non-null  int64  
 5   club_joined_date                161583 non-null  int64  
 6   nationality_name                161583 non-null  int64  
 7   preferred_foot                  161583 non-null  int64  
 8   work_rate                       161583 non-null  int64  
 9   body_type                       161583 non-null  int64  
 10  ls                              161583 non-null  int64  
 11  st                              161583 non-null  int64  
 12  rs              

In [488]:
# Finding the best features 
feature_select = SelectKBest(score_func= f_regression, k=15)

In [489]:
#fitting and transforming the feature select on the training data 
best_features = feature_select.fit_transform(X,Y)

In [490]:
important_features = pd.DataFrame({'Features': X.columns, 'Scores': feature_select.scores_})
important_features_subset = important_features.nlargest(15,'Scores')['Features'].values
important_features_subset

array(['movement_reactions', 'potential', 'passing', 'wage_eur',
       'mentality_composure', 'value_eur', 'dribbling',
       'attacking_short_passing', 'mentality_vision',
       'international_reputation', 'skill_long_passing',
       'power_shot_power', 'physic', 'age', 'skill_ball_control'],
      dtype=object)

In [491]:
X = cleaned_data[important_features_subset]

X.head()

,movement_reactions,potential,passing,wage_eur,mentality_composure,value_eur,dribbling,attacking_short_passing,mentality_vision,international_reputation,skill_long_passing,power_shot_power,physic,age,skill_ball_control
0,94,95,86.000000,550000.0,57.816892,100500000.0,96.000000,89,90,5,76,80,63.000000,27,96
1,90,92,81.000000,375000.0,57.816892,79000000.0,91.000000,82,81,5,72,94,79.000000,29,92
2,89,90,83.000000,275000.0,57.816892,54500000.0,92.000000,86,84,5,76,86,64.000000,30,90
3,85,90,81.000000,275000.0,57.816892,52500000.0,86.000000,84,83,5,76,93,86.000000,32,90
4,89,90,56.938175,300000.0,57.816892,63500000.0,62.081872,42,20,5,41,42,64.814364,28,31


In [492]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

# Saving the scaler to a file
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

scaled_X = pd.DataFrame(scaled_X,columns = X.columns)
scaled_X.head()

,movement_reactions,potential,passing,wage_eur,mentality_composure,value_eur,dribbling,attacking_short_passing,mentality_vision,international_reputation,skill_long_passing,power_shot_power,physic,age,skill_ball_control
0,3.535775,3.875315,2.940185e+00,24.706815,6.456683e-16,16.451449,3.492538e+00,2.086204,2.588430,10.148137,1.536683,1.499832,-1.976505e-01,0.401872,2.281310
1,3.099571,3.396013,2.434335e+00,16.687273,6.456683e-16,12.848571,2.977689e+00,1.609554,1.957672,10.148137,1.273977,2.401757,1.545335e+00,0.830120,2.039859
2,2.990520,3.076478,2.636675e+00,12.104678,6.456683e-16,8.742966,3.080659e+00,1.881925,2.167924,10.148137,1.536683,1.886371,-8.871396e-02,1.044244,1.919133
3,2.554317,3.076478,2.434335e+00,12.104678,6.456683e-16,8.407815,2.462841e+00,1.745740,2.097840,10.148137,1.536683,2.337334,2.307891e+00,1.472491,1.919133
4,2.990520,3.076478,-7.188561e-16,13.250326,6.456683e-16,10.251147,7.316434e-16,-1.114157,-2.317466,10.148137,-0.761996,-0.948252,-1.548082e-15,0.615996,-1.642271


# 3 Training Models: Created and trained with cross-validation either RandomForest, XGBoost, Gradient Boost Regressors that can predict a player rating.values

In [493]:
# Splitting the data 
X_train,X_test,Y_train,Y_test = train_test_split(scaled_X, Y, test_size=0.3, random_state=42)
print("Running")

Running


In [494]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')

# fiting and transforming the imputer on our data 
X_train_impute = imputer.fit_transform(X_train)
X_test_impute = imputer.fit_transform(X_test)
print("Running")

Running


In [495]:
# Train a LinearRegression model
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train_impute, Y_train)
print("Running")

Running


In [496]:
randomForest_model = RandomForestRegressor(n_estimators=100, random_state=42)
randomForest_model.fit(X_train_impute,Y_train)
print("Running")

Running


In [497]:
# Train an XGBRegressor
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror')
xgboost_model.fit(X_train_impute, Y_train)
print("Running")

Running


In [498]:
# Evaluation: Used MAE or RMSE and then fine tuned model, train and tested it again.

In [499]:
gradientBoost_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gradientBoost_model.fit(X_train_impute, Y_train)
print("Running")

Running


In [500]:
# Performing k-fold cross-validation for models trained 
# Attaching ives to make the values positive

randomForest_cv_scores = cross_val_score(randomForest_model, X_train_impute, Y_train, cv=5, scoring='neg_mean_squared_error')
random_forest_rmse = sqrt(-randomForest_cv_scores.mean())

xgboost_cv_scores = cross_val_score(xgboost_model, X_train_impute, Y_train, cv=5, scoring='neg_mean_squared_error')
xgboost_rmse = sqrt(-xgboost_cv_scores.mean())

gradientBoost_cv_scores = cross_val_score(gradientBoost_model, X_train_impute, Y_train, cv=5, scoring='neg_mean_squared_error')
gradient_boost_rmse = sqrt(-gradientBoost_cv_scores.mean())
print("Running")

Running


In [501]:
# Print the cross-validated RMSE for each model
print("Cross-Validated RMSE for Random Forest model:", random_forest_rmse)
print("Cross-Validated RMSE for XGBoost model:", xgboost_rmse)
print("Cross-Validated RMSE for Gradient Boosting model:", gradient_boost_rmse)

Cross-Validated RMSE for Random Forest model: 0.8665061417495599
Cross-Validated RMSE for XGBoost model: 0.9368672915509101
Cross-Validated RMSE for Gradient Boosting model: 1.195820374068687


In [502]:
# Fitting and transforming the testing data
X_test_impute = imputer.fit_transform(X_test)
random_predict = randomForest_model.predict(X_test_impute)
print(random_predict)

[61.67 63.66 68.34 ... 60.43 53.43 62.33]


In [503]:
linear_predict = linear_regression_model.predict(X_test_impute)
print(linear_predict)

[64.36036488 61.23829619 67.11539646 ... 60.13619807 56.36202955
 62.76384734]


In [504]:
xg_predict = xgboost_model.predict(X_test_impute)
print(xg_predict)

[63.902576 63.22243  68.32214  ... 60.225594 52.501575 61.22226 ]


In [505]:
gradient_predict = gradientBoost_model.predict(X_test_impute)
print(gradient_predict)

[62.4625871  63.74701448 68.5621515  ... 59.56202768 53.05589931
 61.48905003]


In [506]:
#Evaluating the models using the MAE and RMSE
def evaluate_model_performance(y_true, y_pred, model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    performance_metrics = {
        "Model": model_name,
        "Mean Absolute Error": mae,
        "Root Mean Squared Error": rmse,
        "R² Score": r2
    }
    
    return performance_metrics

In [507]:
#Finding each model's performance 
random_forest_performance = evaluate_model_performance(Y_test, random_predict, model_name="Random Forest")
xgboost_performance = evaluate_model_performance(Y_test, xg_predict, model_name="XGBoost")
gradient_boosting_performance = evaluate_model_performance(Y_test, gradient_predict, model_name="Gradient Boosting Regressor")


In [508]:
# Print the performance metrics
print('Running')
for model_performance in [random_forest_performance,gradient_boosting_performance,gradient_boosting_performance]:
     print(f"{model_performance['Model']} Performance:")
     print(f"Mean Absolute Error: {model_performance['Mean Absolute Error']}")
     print(f"Root Mean Squared Error: {model_performance['Root Mean Squared Error']}")
     print(f"R² Score: {model_performance['R² Score']}")
     print()

Running
Gradient Boosting Regressor Performance:
Mean Absolute Error: 0.9040330102782256
Root Mean Squared Error: 1.1890324841783662
R² Score: 0.9713940453837939



In [509]:
# Define the parameter grids
param_grid = {
    'n_estimators': [200, 300, 400],
    'max_depth': [3, 4, 5]
}

In [510]:
# Perform GridSearchCV for Random Forest
rf_grid_search = GridSearchCV(estimator=randomForest_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
rf_grid_search.fit(X_train_impute, Y_train)
print(f"Best parameters for Random Forest: {rf_grid_search.best_params_}")
print(f"Best score for Random Forest: {rf_grid_search.best_score_}")


Best parameters for Random Forest: {'max_depth': 5, 'n_estimators': 400}
Best score for Random Forest: -3.3938313867568923


In [511]:
# # Perform GridSearchCV for XGBoost
xgb_grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
xgb_grid_search.fit(X_train_impute, Y_train)
print(f"Best parameters for XGBoost: {xgb_grid_search.best_params_}")
print(f"Best score for XGBoost: {xgb_grid_search.best_score_}")

Best parameters for XGBoost: {'max_depth': 5, 'n_estimators': 400}
Best score for XGBoost: -0.8191430598850566


In [512]:
# # Perform GridSearchCV for Gradient Boosting
gb_grid_search = GridSearchCV(estimator=gradientBoost_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
gb_grid_search.fit(X_train_impute, Y_train)
print(f"Best parameters for Gradient Boosting: {gb_grid_search.best_params_}")
print(f"Best score for Gradient Boosting: {gb_grid_search.best_score_}")

Best parameters for Gradient Boosting: {'max_depth': 5, 'n_estimators': 400}
Best score for Gradient Boosting: -0.8382619694662539


In [513]:
# Use the best estimators to make predictions
print('runing')
best_rf_model = rf_grid_search.best_estimator_
best_xgb_model = xgb_grid_search.best_estimator_
best_gb_model = gb_grid_search.best_estimator_

rf_predictions = best_rf_model.predict(X_test_impute)
xgb_predictions = best_xgb_model.predict(X_test_impute)
gb_predictions = best_gb_model.predict(X_test_impute)


runing


In [514]:
# Evaluate the models using the previously defined function
rf_performance = evaluate_model_performance(Y_test, rf_predictions, model_name="Random Forest")
xgb_performance = evaluate_model_performance(Y_test, xgb_predictions, model_name="XGBoost")
gb_performance = evaluate_model_performance(Y_test, gb_predictions, model_name="Gradient Boosting")

In [515]:
# Print the performance metrics
for performance in [xgb_performance,rf_performance,gb_performance]:
    print(f"{performance['Model']} Performance:")
    print(f"Mean Absolute Error: {performance['Mean Absolute Error']}")
    print(f"Root Mean Squared Error: {performance['Root Mean Squared Error']}")
    print(f"R² Score: {performance['R² Score']}")
    print()

Random Forest Performance:
Mean Absolute Error: 1.3830973488737341
Root Mean Squared Error: 1.8319314903432493
R² Score: 0.9320972345116846



In [ ]:
import pickle

# Define and train the XGBoost Regressor model with the best hyperparameters
best_model = GradientBoostingRegressor(max_depth=5, n_estimators=400, random_state=42)
best_model.fit(X_train_impute, Y_train)


# Saving the trained model to a file
with open('grad_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)


# Test with new data set: Used the data(players_22) to test how good is the model with completely new data.

In [ ]:
# Load and preprocess training data
test_data = pd.read_csv("players_22 (1).csv")
cleaned_tdata = preprocess_data(test_data)

y = cleaned_tdata['overall']
x = cleaned_tdata[important_features_subset]
x.info()

In [ ]:
important_features_subset

In [ ]:
#Fitting and Transforming the data 
x_scaled = scaler.fit_transform(x)
x_scaled = pd.DataFrame(x_scaled, columns=x.columns)

In [ ]:
#Checking for any columns that might have all zeros
all_zero_columns = cleaned_data.columns[(cleaned_data == 0).all()]
print("Columns with all zeros:", all_zero_columns)

In [ ]:
#Checking for any columns that might have all nans
nan_columns = cleaned_data.columns[cleaned_data.isna().any()]
print("Columns with NaN values:", nan_columns)

In [ ]:
# filename = 'best_gradient_boost_model.pkl'

with open('best_xgboost_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)


In [ ]:
# Fit and transform the imputer on testing data
xtest_imputed = imputer.fit_transform(x_scaled)

In [ ]:
#Making Prediction
new_pred = loaded_model.predict(xtest_imputed)

In [ ]:
# Calculate evaluation metrics
mae = mean_absolute_error(y, new_pred)
rmse = sqrt(mean_squared_error(y, new_pred))
r2 = r2_score(y, new_pred)

# Print the evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R^2):", r2)
